
---

# 🔒 9. Security Best Practices

## A) Least privilege (default posture)

```dockerfile
# run as non-root
RUN useradd -m app && mkdir /app && chown -R app:app /app
USER app
```

```bash
# drop all caps, add only what you need (eg bind <1024> ports)
docker run --user 1000:1000 --cap-drop ALL --cap-add NET_BIND_SERVICE myapp
```

* ❌ Avoid `--privileged`.
* Prefer **non-root** + **cap-drop**.

---

## B) Lock down runtime

```bash
docker run \
  --read-only --tmpfs /tmp \
  --security-opt no-new-privileges:true \
  --pids-limit 200 --memory 512m --cpus 1 \
  myapp
```

* **read-only FS** + **tmpfs** for writes.
* Limit **PIDs/RAM/CPU**.
* `no-new-privileges` stops escalation.

---

## C) Network hygiene

```bash
docker run --network none myjob        # fully isolated
docker run -p 8080:8080 myapi          # publish only needed ports
```

* Use **user-defined bridge** per app; isolate stacks.
* Don’t expose the Docker socket to containers.

---

## D) Image provenance & size

```bash
# small, pinned base
FROM python:3.11-slim@sha256:<digest>
# keep layers clean
RUN apt-get update && apt-get install -y curl \
 && rm -rf /var/lib/apt/lists/*
```

```bash
export DOCKER_CONTENT_TRUST=1   # verify signed images
docker build --pull -t myapp .  # pull patched bases
```

* Prefer **slim/alpine**, **pin by digest**, **rebuild regularly**.

---

## E) Secrets (never bake into image)

```bash
# dev: env file (keep out of git)
docker run --env-file .env myapp

# prod/orchestrators: use secrets (Swarm/K8s) instead of envs
```

* Add `.env` to **.gitignore**.
* Use mounts for secret files (`:ro`).

---

## F) Scan & audit

```bash
docker scan myimage            # Snyk-backed
# or (popular alternative)
trivy image myimage
```

```bash
# host audit (⚠️ uses high privileges; review flags)
docker run --rm --net host --pid host --privileged \
  -v /var/run/docker.sock:/var/run/docker.sock -v /etc:/etc \
  docker/docker-bench-security
```

---

## G) Supply-chain & deps

* Use **private registries** with access control.
* Pin app deps (`requirements.txt`, `package-lock.json`).
* Verify base images come from **trusted orgs**.

---

## H) Logging & health

```yaml
# Compose snippet
services:
  api:
    image: myapp
    healthcheck:
      test: ["CMD","curl","-f","http://localhost:8080/health"]
      interval: 10s
      retries: 3
```

* Healthchecks catch bad states early.
* Centralize logs; avoid writing secrets to stdout.

---

## Quick checklist

* Non-root ✅  Cap-drop ✅  Read-only FS ✅
* Limit CPU/RAM/PIDs ✅  Minimal, pinned base ✅
* Secrets via env-file/manager ✅  Scan images ✅
* Least-open ports/networks ✅  No Docker socket in containers ✅
